In [ ]:
import sys
sys.path.append('..')
import spacy
import numpy as np

from tools.TextProcessing import nlp
from tools.BasicUtils import my_write, my_read

In [ ]:
def find_dependency_path_from_tree(doc, kw1:spacy.tokens.span.Span, kw2:spacy.tokens.span.Span):
    idx1 = kw1[-1].i
    idx2 = kw2[-1].i
    branch = np.zeros(len(doc))
    i = idx1
    while branch[i] == 0:
        branch[i] = 1
        i = doc[i].head.i
    i = idx2
    while branch[i] != 1:
        branch[i] = 2
        if i == doc[i].head.i:
            return ''
        i = doc[i].head.i
    dep1 = []
    j = idx1
    while j != i:
        dep1.append('i_%s' % doc[j].dep_)
        j = doc[j].head.i
    dep2 = []
    j = idx2
    while j != i:
        dep2.append(doc[j].dep_)
        j = doc[j].head.i
    dep2.reverse()
    if branch[idx2] == 1:
        # kw2 is along the heads of kw1
        return ' '.join(dep1)
    elif i == idx1:
        # kw1 is along the heads of kw2
        return ' '.join(dep2)
    else:
        return ' '.join(dep1 + dep2)

In [ ]:
p2line_dict = {}
sents = my_read('temp_sent.txt')
for i, line in enumerate(sents):
    doc = nlp(line.strip())
    l = [s for s in doc.noun_chunks if s[-1].pos_ != 'PRON']
    if len(l) < 2:
        continue
    for j in range(len(l)-1):
        for k in range(j, len(l)):
            p = find_dependency_path_from_tree(doc, l[j], l[k])
            if not p:
                continue
            if p not in p2line_dict:
                p2line_dict[p] = []
            p2line_dict[p].append({'kw1':str(l[j]), 'kw2':str(l[k]), 'line':i})

In [ ]:
len(p2line_dict)

In [ ]:
freq_p = [k for k, c in p2line_dict.items() if len(c) > 10]

In [ ]:
len(freq_p)

In [ ]:
my_write('freq_path_10.txt', freq_p)

In [ ]:
p = 'prep pobj conj'
my_write(p, ['%s\t\t%s\t\t%s' % (d['kw1'], d['kw2'], sents[d['line']]) for d in p2line_dict[p]])

In [ ]:
s = 'many machine learning tools have been applied to this problem using static machine learning structures such as neural networks or svm that are unable to accommodate new information into their existing models.'
doc = nlp(s)
l = list(doc.noun_chunks)
print(l)
find_dependency_path_from_tree(doc, l[2], l[4])

In [ ]:
type(doc[0:2])

In [ ]:
from tools.BasicUtils import SparseRetrieveSentForPairCoOccur

In [ ]:
sparse_retriever = SparseRetrieveSentForPairCoOccur('../data/corpus/small_sent.txt', 'data/occur.json')